# SECTION 1 - PRE PROCESSING

### Install Turicreate

In [0]:
!pip install turicreate

### Import necessary Libraries

In [0]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import turicreate as tc # Turicreate library for recommendations

### Google Drive Pre Requisite

In [0]:
# Code to read csv file into Colaboratory:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials 

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
link = 'https://drive.google.com/open?id=1dJ2MmNB9_UG3R5HZohSIqqu_dC0_N-S2'

fluff, id = link.split('=')

print (id) # Verify that you have everything after '='

### Read data from the file.

In [5]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile("steam-200k.csv")  
df = pd.read_csv("steam-200k.csv", header=None, index_col=None, names=['UserID', 'Game', 'Action', 'Hours', 'Other'])
df.head()

,UserID,Game,Action,Hours,Other
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


### Check shape of the dataset

In [6]:
df.shape

(200000, 5)

### Check the datatypes of the columns

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 5 columns):
UserID    200000 non-null int64
Game      200000 non-null object
Action    200000 non-null object
Hours     200000 non-null float64
Other     200000 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 7.6+ MB


### Check the values in "Other" column

In [8]:
df["Other"].value_counts()

0    200000
Name: Other, dtype: int64

### Remove the column

In [0]:
df.drop(["Other"],axis=1,inplace = True)

### Statistical Information

In [10]:
df.loc[df["Action"]=="play","Hours"].describe()

count    70489.000000
mean        48.878063
std        229.335236
min          0.100000
25%          1.000000
50%          4.500000
75%         19.100000
max      11754.000000
Name: Hours, dtype: float64

# SECTION 2 - RANKING

### Dataset

In [11]:
df.head()

,UserID,Game,Action,Hours
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


### Groupby using Actions

In [12]:
df_group_values = df.groupby(["UserID","Action"]).size()
df_group_values.head(25)

UserID  Action  
5250    play          6
        purchase     21
76767   play         20
        purchase     36
86540   play         15
        purchase     82
103360  purchase     10
144736  play          1
        purchase      8
181212  play          2
        purchase     12
229911  play         18
        purchase     27
298950  play        175
        purchase    259
299153  purchase     14
381543  play          1
        purchase     10
547685  play          5
        purchase     25
554278  play          9
        purchase     28
561758  play         80
        purchase    148
577614  play          1
dtype: int64

### Describe the dataset

In [13]:
print("Number of unique games : {0}".format(len(df.Game.unique())))
print("Number of unique users : {0}".format(len(df.UserID.unique())))
print("Number of total purchases by users : {0}".format(len(df.loc[df['Action'] == 'purchase'])))
print("Number of total plays by users : {0}".format(len(df.loc[df['Action'] == 'play'])))

Number of unique games : 5155
Number of unique users : 12393
Number of total purchases by users : 129511
Number of total plays by users : 70489


### Create 2 new datasets 

In [0]:
df_purchase =  df.loc[df['Action'] == 'purchase', ['UserID','Game','Action','Hours']].rename(columns={'Hours': 'Purchased'})  # Purchased Games
df_play = df.loc[df['Action'] == 'play', ['UserID','Game','Hours']].rename(columns={'Hours': 'Played_hours'})                 # Played games by hour

In [15]:
print('shape of purchase dataset : ', df_purchase.shape[0])
print('shape of play dataset : ', df_play.shape[0])

shape of purchase dataset :  129511
shape of play dataset :  70489


### Merge datasets

In [0]:
total_dataset = pd.merge(df_purchase, df_play, left_on=['UserID','Game'], right_on=['UserID','Game'], how='left').fillna(0)

In [17]:
total_dataset.head(20)

,UserID,Game,Action,Purchased,Played_hours
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,273.0
1,151603712,Fallout 4,purchase,1.0,87.0
2,151603712,Spore,purchase,1.0,14.9
3,151603712,Fallout New Vegas,purchase,1.0,12.1
4,151603712,Left 4 Dead 2,purchase,1.0,8.9
5,151603712,HuniePop,purchase,1.0,8.5
6,151603712,Path of Exile,purchase,1.0,8.1
7,151603712,Poly Bridge,purchase,1.0,7.5
8,151603712,Left 4 Dead,purchase,1.0,3.3
9,151603712,Team Fortress 2,purchase,1.0,2.8


### Create a ranking system for games per user ID

In [0]:
#Ranking = lambda x: pd.qcut(x,10,labels=False,duplicates='drop')
#total_dataset['Ranking'] = total_dataset.groupby('UserID')['Played_hours'].apply(Ranking)

In [0]:
total_dataset['Ranking'] = total_dataset.groupby('UserID')['Played_hours'].transform(pd.cut, 10, labels=False,duplicates = "drop")+1.0

In [20]:
total_dataset.head(30)

,UserID,Game,Action,Purchased,Played_hours,Ranking
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,273.0,10.0
1,151603712,Fallout 4,purchase,1.0,87.0,4.0
2,151603712,Spore,purchase,1.0,14.9,1.0
3,151603712,Fallout New Vegas,purchase,1.0,12.1,1.0
4,151603712,Left 4 Dead 2,purchase,1.0,8.9,1.0
5,151603712,HuniePop,purchase,1.0,8.5,1.0
6,151603712,Path of Exile,purchase,1.0,8.1,1.0
7,151603712,Poly Bridge,purchase,1.0,7.5,1.0
8,151603712,Left 4 Dead,purchase,1.0,3.3,1.0
9,151603712,Team Fortress 2,purchase,1.0,2.8,1.0


In [21]:
total_dataset.loc[total_dataset["UserID"]==151603712].head(100)

,UserID,Game,Action,Purchased,Played_hours,Ranking
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,273.0,10.0
1,151603712,Fallout 4,purchase,1.0,87.0,4.0
2,151603712,Spore,purchase,1.0,14.9,1.0
3,151603712,Fallout New Vegas,purchase,1.0,12.1,1.0
4,151603712,Left 4 Dead 2,purchase,1.0,8.9,1.0
5,151603712,HuniePop,purchase,1.0,8.5,1.0
6,151603712,Path of Exile,purchase,1.0,8.1,1.0
7,151603712,Poly Bridge,purchase,1.0,7.5,1.0
8,151603712,Left 4 Dead,purchase,1.0,3.3,1.0
9,151603712,Team Fortress 2,purchase,1.0,2.8,1.0


### Create a clean dataset by dropping redundant columns

In [22]:
clean_df = total_dataset.drop(["Purchased","Action"],axis = 1)
clean_df.head(20)

,UserID,Game,Played_hours,Ranking
0,151603712,The Elder Scrolls V Skyrim,273.0,10.0
1,151603712,Fallout 4,87.0,4.0
2,151603712,Spore,14.9,1.0
3,151603712,Fallout New Vegas,12.1,1.0
4,151603712,Left 4 Dead 2,8.9,1.0
5,151603712,HuniePop,8.5,1.0
6,151603712,Path of Exile,8.1,1.0
7,151603712,Poly Bridge,7.5,1.0
8,151603712,Left 4 Dead,3.3,1.0
9,151603712,Team Fortress 2,2.8,1.0


### Display top 5 users based on number of games

In [23]:
clean_df["UserID"].value_counts()

62990992     1075
33865373      785
30246419      766
58345543      667
76892907      597
             ... 
149194171       1
207945140       1
130315685       1
282733934       1
214618086       1
Name: UserID, Length: 12393, dtype: int64

# SECTION 3 - RECOMMENDATION SYSTEM - FACTORIZATION RECOMMENDER

### Create a SFRame from the clean dataframe

In [0]:
dfS = tc.SFrame(clean_df)

### Factorization Recommender System


In [25]:
m = tc.factorization_recommender.create(dfS,
			                              user_id='UserID',
                                    item_id='Game',
                                    target='Ranking')

Preparing data set.

Data has 129535 observations with 12393 users and 5155 items.

Data prepared in: 0.192161s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 3.44582

Final training RMSE: 1.85629

In [26]:
recommendations = m.recommend()

recommendations finished on 1000/12393 queries. users per second: 5341.37

recommendations finished on 2000/12393 queries. users per second: 5290.7

recommendations finished on 3000/12393 queries. users per second: 4944.18

recommendations finished on 4000/12393 queries. users per second: 4989.03

recommendations finished on 5000/12393 queries. users per second: 4968.02

...

recommendations finished on 8000/12393 queries. users per second: 5053.57

recommendations finished on 9000/12393 queries. users per second: 5081.89

recommendations finished on 10000/12393 queries. users per second: 5088.5

recommendations finished on 11000/12393 queries. users per second: 5122.45

recommendations finished on 12000/12393 queries. users per second: 5149.67

### USER DESCRIPTION AND RECOMMENDATIONS

In [27]:
r1 = m.recommend(users=['151603712'])
r1.head(20)

UserID,Game,score,rank
151603712,Counter-Strike,3.5592457665675443,1
151603712,Counter-Strike Source,3.0549562214487356,2
151603712,Counter-Strike GlobalOffensive ...,2.8393320678346914,3
151603712,Unturned,2.8099341406100553,4
151603712,Half-Life 2,2.764833824204759,5
151603712,APB Reloaded,2.7265874272757333,6
151603712,Sid Meier's CivilizationIV Warlords ...,2.6784335984462064,7
151603712,Call of Duty ModernWarfare 2 ...,2.6439070566170972,8
151603712,Tom Clancy's Ghost ReconPhantoms - EU ...,2.6401494382494253,9
151603712,Trove,2.628061128275947,10


In [28]:
clean_df.loc[clean_df["UserID"] == 151603712,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
0,The Elder Scrolls V Skyrim,273.0,10.0
1,Fallout 4,87.0,4.0
2,Spore,14.9,1.0
3,Fallout New Vegas,12.1,1.0
4,Left 4 Dead 2,8.9,1.0
5,HuniePop,8.5,1.0
6,Path of Exile,8.1,1.0
7,Poly Bridge,7.5,1.0
8,Left 4 Dead,3.3,1.0
9,Team Fortress 2,2.8,1.0


In [29]:
r2 = m.recommend(users=['62990992'])
r2

UserID,Game,score,rank
62990992,Unturned,2.64632952349122,1
62990992,APB Reloaded,2.645079465078668,2
62990992,Call of Duty Black Ops,2.5693408711427015,3
62990992,Call of Duty ModernWarfare 2 ...,2.5586390136354726,4
62990992,South Park The Stick ofTruth ...,2.4698157733851236,5
62990992,The Sims(TM) 3,2.419590772556619,6
62990992,H1Z1,2.415520738306002,7
62990992,Call of Duty ModernWarfare 3 ...,2.4082047813185614,8
62990992,Tom Clancy's Ghost ReconPhantoms - NA ...,2.3987765921824735,9
62990992,Train Simulator,2.3931012707376045,10


In [30]:
clean_df.loc[clean_df["UserID"] == 62990992,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
77076,Counter-Strike Global Offensive,663.0,10.0
77077,Sid Meier's Civilization V,550.0,9.0
77078,Total War SHOGUN 2,212.0,4.0
77079,Total War ROME II - Emperor Edition,198.0,3.0
77080,Dungeon Defenders,195.0,3.0
...,...,...,...
78146,Xenophage,0.0,1.0
78147,Xpand Rally Xtreme,0.0,1.0
78148,Zen Bound 2,0.0,1.0
78149,Zombie Driver HD Apocalypse Pack,0.0,1.0


In [31]:
r3 = m.recommend(users=['33865373'])
r3

UserID,Game,score,rank
33865373,Counter-Strike Source,3.097394069152431,1
33865373,Unturned,2.9036603285544675,2
33865373,Counter-Strike GlobalOffensive ...,2.8721647015386385,3
33865373,APB Reloaded,2.753906109797315,4
33865373,Call of Duty ModernWarfare 2 ...,2.725173191594438,5
33865373,Call of Duty Black Ops,2.724220921712474,6
33865373,Trove,2.665265976878361,7
33865373,Tom Clancy's Ghost ReconPhantoms - EU ...,2.654098128395633,8
33865373,Robocraft,2.5854312642091077,9
33865373,South Park The Stick ofTruth ...,2.583168634640531,10


In [32]:
clean_df.loc[clean_df["UserID"] == 33865373,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
47739,The Elder Scrolls V Skyrim,345.0,10.0
47740,The Elder Scrolls IV Oblivion,239.0,7.0
47741,Fallout New Vegas,198.0,6.0
47742,Sid Meier's Civilization IV,135.0,4.0
47743,Sid Meier's Civilization IV,2.0,1.0
...,...,...,...
48519,Xenophage,0.0,1.0
48520,Xotic,0.0,1.0
48521,Zafehouse Diaries,0.0,1.0
48522,Zen Bound 2,0.0,1.0


In [33]:
r4 = m.recommend(users=['30246419'])
r4

UserID,Game,score,rank
30246419,Dota 2,5.207998075949506,1
30246419,Counter-Strike,4.729445227848844,2
30246419,Team Fortress 2,4.72708864273365,3
30246419,Counter-Strike Source,4.145514049994306,4
30246419,Counter-Strike GlobalOffensive ...,4.0367728112810415,5
30246419,APB Reloaded,3.853941184895591,6
30246419,Garry's Mod,3.8401527269357008,7
30246419,Tom Clancy's Ghost ReconPhantoms - EU ...,3.8387627093666357,8
30246419,Trove,3.7580112930932206,9
30246419,South Park The Stick ofTruth ...,3.7073536901348394,10


In [34]:
clean_df.loc[clean_df["UserID"] == 30246419,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
62101,The Witcher 3 Wild Hunt,99.0,10.0
62102,Fallout 4,97.0,10.0
62103,Two Worlds II,61.0,7.0
62104,Test Drive Unlimited 2,57.0,6.0
62105,Middle-earth Shadow of Mordor,53.0,6.0
...,...,...,...
62862,Yury,0.0,1.0
62863,Z,0.0,1.0
62864,Zafehouse Diaries,0.0,1.0
62865,Zeno Clash 2,0.0,1.0


In [35]:
r5 = m.recommend(users=['58345543'])
r5

UserID,Game,score,rank
58345543,Counter-Strike,5.893600480126695,1
58345543,Sid Meier's CivilizationIV Warlords ...,5.13283302025731,2
58345543,South Park The Stick ofTruth ...,5.040564761924104,3
58345543,Call of Duty Black Ops,4.949698524045305,4
58345543,Empire Total War,4.834535922583671,5
58345543,Wolfenstein The New Order,4.829068647193269,6
58345543,H1Z1,4.819721035155014,7
58345543,The Forest,4.8165560825341505,8
58345543,Napoleon Total War,4.812374371530966,9
58345543,Batman Arkham Origins,4.811370683329658,10


In [36]:
clean_df.loc[clean_df["UserID"] == 58345543,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
115598,Spiral Knights,70.0,10.0
115599,Blacklight Retribution,64.0,10.0
115600,Dungeon Defenders,56.0,8.0
115601,Terraria,54.0,8.0
115602,Borderlands 2,53.0,8.0
...,...,...,...
116260,Wrath of Athena,0.0,1.0
116261,WTFast Gamers Private Network (GPN),0.0,1.0
116262,Xam,0.0,1.0
116263,You Have to Win the Game,0.0,1.0


In [37]:
clean_df.loc[clean_df["UserID"] == 58345543,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
115598,Spiral Knights,70.0,10.0
115599,Blacklight Retribution,64.0,10.0
115600,Dungeon Defenders,56.0,8.0
115601,Terraria,54.0,8.0
115602,Borderlands 2,53.0,8.0
...,...,...,...
116260,Wrath of Athena,0.0,1.0
116261,WTFast Gamers Private Network (GPN),0.0,1.0
116262,Xam,0.0,1.0
116263,You Have to Win the Game,0.0,1.0


# SECTION 4 - RECOMMENDATION SYSTEM - RANKING FACTORIZATION RECOMMENDER

### Ranking Factorization Recommender System

In [38]:
n = tc.ranking_factorization_recommender.create(dfS,
			                              user_id='UserID',
                                    item_id='Game',
                                    target='Ranking')


Preparing data set.

Data has 129535 observations with 12393 users and 5155 items.

Data prepared in: 0.19517s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 6.76256

Final training RMSE: 1.8146

In [39]:
recommendations = n.recommend()

recommendations finished on 1000/12393 queries. users per second: 2130.87

recommendations finished on 2000/12393 queries. users per second: 2145.51

recommendations finished on 3000/12393 queries. users per second: 2159.97

recommendations finished on 4000/12393 queries. users per second: 2174.46

recommendations finished on 5000/12393 queries. users per second: 2171.34

...

recommendations finished on 8000/12393 queries. users per second: 2181.27

recommendations finished on 9000/12393 queries. users per second: 2183.11

recommendations finished on 10000/12393 queries. users per second: 2181.84

recommendations finished on 11000/12393 queries. users per second: 2184.74

recommendations finished on 12000/12393 queries. users per second: 2181.4

### USER DESCRIPTION AND RECOMMENDATIONS

In [40]:
r1 = n.recommend(users=['151603712'])
r1.head(20)

UserID,Game,score,rank
151603712,Counter-Strike,2.9943991838568014,1
151603712,Unturned,2.8328644795888227,2
151603712,Counter-Strike GlobalOffensive ...,2.7636822401159566,3
151603712,Counter-Strike Source,2.7366351498835844,4
151603712,Half-Life 2,2.201855817305402,5
151603712,Terraria,2.0601573898070615,6
151603712,Sid Meier's CivilizationV ...,2.0234077064865392,7
151603712,Call of Duty ModernWarfare 2 ...,1.9977704449170393,8
151603712,Half-Life 2 Deathmatch,1.9747004969590467,9
151603712,Portal 2,1.9594174935096067,10


In [41]:
clean_df.loc[clean_df["UserID"] == 151603712,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
0,The Elder Scrolls V Skyrim,273.0,10.0
1,Fallout 4,87.0,4.0
2,Spore,14.9,1.0
3,Fallout New Vegas,12.1,1.0
4,Left 4 Dead 2,8.9,1.0
5,HuniePop,8.5,1.0
6,Path of Exile,8.1,1.0
7,Poly Bridge,7.5,1.0
8,Left 4 Dead,3.3,1.0
9,Team Fortress 2,2.8,1.0


In [42]:
r2 = n.recommend(users=['62990992'])
r2

UserID,Game,score,rank
62990992,Unturned,1.6684413565867704,1
62990992,Call of Duty Black Ops,0.9154417826407713,2
62990992,Call of Duty ModernWarfare 2 - Multiplayer ...,0.8093712343447965,3
62990992,Rust,0.7000561370127958,4
62990992,Grand Theft Auto V,0.69807136194642,5
62990992,PAYDAY 2,0.6848481311076444,6
62990992,Fallout New Vegas,0.6587077392810148,7
62990992,Arma 2 OperationArrowhead ...,0.5799240185492796,8
62990992,Don't Starve,0.5527192963832182,9
62990992,Football Manager 2014,0.5204056216948789,10


In [43]:
clean_df.loc[clean_df["UserID"] == 62990992,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
77076,Counter-Strike Global Offensive,663.0,10.0
77077,Sid Meier's Civilization V,550.0,9.0
77078,Total War SHOGUN 2,212.0,4.0
77079,Total War ROME II - Emperor Edition,198.0,3.0
77080,Dungeon Defenders,195.0,3.0
...,...,...,...
78146,Xenophage,0.0,1.0
78147,Xpand Rally Xtreme,0.0,1.0
78148,Zen Bound 2,0.0,1.0
78149,Zombie Driver HD Apocalypse Pack,0.0,1.0


In [44]:
r3 = n.recommend(users=['33865373'])
r3

UserID,Game,score,rank
33865373,Counter-Strike GlobalOffensive ...,2.7974024428599638,1
33865373,Counter-Strike Source,2.5850897146933836,2
33865373,Unturned,2.144098819660501,3
33865373,Sid Meier's CivilizationV ...,1.8520054532759946,4
33865373,Trove,1.5775069607966703,5
33865373,Robocraft,1.5251051201098722,6
33865373,Call of Duty ModernWarfare 2 - Multiplayer ...,1.4633719040625852,7
33865373,Age of Empires II HDEdition ...,1.4427422894709867,8
33865373,Call of Duty Black Ops,1.4271529926532072,9
33865373,Borderlands 2,1.4221416129344266,10


In [45]:
clean_df.loc[clean_df["UserID"] == 33865373,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
47739,The Elder Scrolls V Skyrim,345.0,10.0
47740,The Elder Scrolls IV Oblivion,239.0,7.0
47741,Fallout New Vegas,198.0,6.0
47742,Sid Meier's Civilization IV,135.0,4.0
47743,Sid Meier's Civilization IV,2.0,1.0
...,...,...,...
48519,Xenophage,0.0,1.0
48520,Xotic,0.0,1.0
48521,Zafehouse Diaries,0.0,1.0
48522,Zen Bound 2,0.0,1.0


In [46]:
r4 = n.recommend(users=['30246419'])
r4

UserID,Game,score,rank
30246419,Dota 2,4.95794233070548,1
30246419,Team Fortress 2,3.9674983857744497,2
30246419,Counter-Strike,3.667633691298322,3
30246419,Counter-Strike Source,3.5237879394167226,4
30246419,Counter-Strike GlobalOffensive ...,3.469168865548925,5
30246419,Unturned,3.2106282948129934,6
30246419,Garry's Mod,3.014220976697759,7
30246419,Grand Theft Auto V,2.662679785358266,8
30246419,Call of Duty ModernWarfare 2 - Multiplayer ...,2.645735749470548,9
30246419,Robocraft,2.6454599140756887,10


In [47]:
clean_df.loc[clean_df["UserID"] == 30246419,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
62101,The Witcher 3 Wild Hunt,99.0,10.0
62102,Fallout 4,97.0,10.0
62103,Two Worlds II,61.0,7.0
62104,Test Drive Unlimited 2,57.0,6.0
62105,Middle-earth Shadow of Mordor,53.0,6.0
...,...,...,...
62862,Yury,0.0,1.0
62863,Z,0.0,1.0
62864,Zafehouse Diaries,0.0,1.0
62865,Zeno Clash 2,0.0,1.0


In [48]:
r5 = n.recommend(users=['58345543'])
r5

UserID,Game,score,rank
58345543,Counter-Strike,3.296862529086427,1
58345543,Empire Total War,2.37910200791295,2
58345543,Euro Truck Simulator 2,2.343964145945863,3
58345543,Grand Theft Auto V,2.326126561927156,4
58345543,Age of Empires II HDEdition ...,2.303986863660173,5
58345543,Counter-Strike ConditionZero ...,2.2539179040664,6
58345543,Call of Duty Black Ops,2.176072941588716,7
58345543,H1Z1,2.1379707618230146,8
58345543,Call of Duty ModernWarfare 3 - Multiplayer ...,2.097364903854684,9
58345543,Napoleon Total War,2.0804790122979444,10


In [49]:
clean_df.loc[clean_df["UserID"] == 58345543,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
115598,Spiral Knights,70.0,10.0
115599,Blacklight Retribution,64.0,10.0
115600,Dungeon Defenders,56.0,8.0
115601,Terraria,54.0,8.0
115602,Borderlands 2,53.0,8.0
...,...,...,...
116260,Wrath of Athena,0.0,1.0
116261,WTFast Gamers Private Network (GPN),0.0,1.0
116262,Xam,0.0,1.0
116263,You Have to Win the Game,0.0,1.0


# SECTION 5 - RECOMMENDATION SYSTEM - ITEM SIMILARITY RECOMMENDER

### Item Similarity Recommender System

In [50]:
o = tc.item_similarity_recommender.create(dfS,
			                              user_id='UserID',
                                    item_id='Game',
                                    target='Ranking')

Warning: Ignoring columns Played_hours;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 129535 observations with 12393 users and 5155 items.

Data prepared in: 0.161111s

...

| 590.943ms                           | 100              | 5155            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.618911s

In [51]:
recommendations = o.recommend()

recommendations finished on 1000/12393 queries. users per second: 17506.1

recommendations finished on 2000/12393 queries. users per second: 17815.6

recommendations finished on 3000/12393 queries. users per second: 17982.5

recommendations finished on 4000/12393 queries. users per second: 17581.7

recommendations finished on 5000/12393 queries. users per second: 17692.8

...

recommendations finished on 8000/12393 queries. users per second: 17124.6

recommendations finished on 9000/12393 queries. users per second: 17270.8

recommendations finished on 10000/12393 queries. users per second: 17440

recommendations finished on 11000/12393 queries. users per second: 17571.6

recommendations finished on 12000/12393 queries. users per second: 17723.1

### USER DESCRIPTION AND RECOMMENDATIONS

In [52]:
r1 = o.recommend(users=['151603712'])
r1.head(20)

UserID,Game,score,rank
151603712,Borderlands 2,0.08844002187252045,1
151603712,The Witcher 2 Assassinsof Kings Enhanced Edi ...,0.08657304048538209,2
151603712,Portal 2,0.08542347103357315,3
151603712,Metro 2033,0.08315240740776061,4
151603712,Dishonored,0.0812200978398323,5
151603712,Batman Arkham City GOTY,0.07957937866449356,6
151603712,Company of Heroes (NewSteam Version) ...,0.07412561774253845,7
151603712,Company of Heroes,0.06891263723373413,8
151603712,Batman Arkham Asylum GOTYEdition ...,0.0677080437541008,9
151603712,Deus Ex Human Revolution,0.06749018281698227,10


In [53]:
clean_df.loc[clean_df["UserID"] == 151603712,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
0,The Elder Scrolls V Skyrim,273.0,10.0
1,Fallout 4,87.0,4.0
2,Spore,14.9,1.0
3,Fallout New Vegas,12.1,1.0
4,Left 4 Dead 2,8.9,1.0
5,HuniePop,8.5,1.0
6,Path of Exile,8.1,1.0
7,Poly Bridge,7.5,1.0
8,Left 4 Dead,3.3,1.0
9,Team Fortress 2,2.8,1.0


In [54]:
r2 = o.recommend(users=['62990992'])
r2

UserID,Game,score,rank
62990992,Brtal Legend,0.021146394507715317,1
62990992,Overlord Raising Hell,0.02084848859560177,2
62990992,Mark of the Ninja,0.020262867937820235,3
62990992,Bastion,0.01905287171570996,4
62990992,Darksiders,0.018422954928562437,5
62990992,Dishonored,0.015959320387590244,6
62990992,LIMBO,0.014379451616426533,7
62990992,FTL Faster Than Light,0.014343305585089694,8
62990992,Rise of the Argonauts,0.013882566927077618,9
62990992,The Walking Dead,0.013650716299867808,10


In [55]:
clean_df.loc[clean_df["UserID"] == 62990992,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
77076,Counter-Strike Global Offensive,663.0,10.0
77077,Sid Meier's Civilization V,550.0,9.0
77078,Total War SHOGUN 2,212.0,4.0
77079,Total War ROME II - Emperor Edition,198.0,3.0
77080,Dungeon Defenders,195.0,3.0
...,...,...,...
78146,Xenophage,0.0,1.0
78147,Xpand Rally Xtreme,0.0,1.0
78148,Zen Bound 2,0.0,1.0
78149,Zombie Driver HD Apocalypse Pack,0.0,1.0


In [56]:
r3 = o.recommend(users=['33865373'])
r3

UserID,Game,score,rank
33865373,GoD Factory Wingmen,0.021196801407435507,1
33865373,Guacamelee! Gold Edition,0.020821821290193145,2
33865373,The Swapper,0.018927239956929513,3
33865373,Titan Quest,0.018739960012362174,4
33865373,Gray Matter,0.01720337124214959,5
33865373,Orcs Must Die! 2,0.01719852114461132,6
33865373,The Witcher 2 Assassinsof Kings Enhanced Edi ...,0.016778717182346227,7
33865373,The Walking Dead,0.01654549572885651,8
33865373,Overlord Raising Hell,0.01589132845401764,9
33865373,Shank 2,0.01569922314476721,10


In [57]:
clean_df.loc[clean_df["UserID"] == 33865373,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
47739,The Elder Scrolls V Skyrim,345.0,10.0
47740,The Elder Scrolls IV Oblivion,239.0,7.0
47741,Fallout New Vegas,198.0,6.0
47742,Sid Meier's Civilization IV,135.0,4.0
47743,Sid Meier's Civilization IV,2.0,1.0
...,...,...,...
48519,Xenophage,0.0,1.0
48520,Xotic,0.0,1.0
48521,Zafehouse Diaries,0.0,1.0
48522,Zen Bound 2,0.0,1.0


In [58]:
r4 = o.recommend(users=['30246419'])
r4

UserID,Game,score,rank
30246419,Sine Mora,0.023019409069744808,1
30246419,Dishonored,0.02135214456927635,2
30246419,Dear Esther,0.019610821494000702,3
30246419,BioShock Infinite,0.01912367100458057,4
30246419,Gratuitous Space Battles,0.017178096868618046,5
30246419,Intrusion 2,0.01690423143082771,6
30246419,Osmos,0.01651441509073431,7
30246419,Proteus,0.01542679405966295,8
30246419,Red Faction GuerrillaSteam Edition ...,0.015398942670332113,9
30246419,Thomas Was Alone,0.014901416417952425,10


In [59]:
clean_df.loc[clean_df["UserID"] == 30246419,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
62101,The Witcher 3 Wild Hunt,99.0,10.0
62102,Fallout 4,97.0,10.0
62103,Two Worlds II,61.0,7.0
62104,Test Drive Unlimited 2,57.0,6.0
62105,Middle-earth Shadow of Mordor,53.0,6.0
...,...,...,...
62862,Yury,0.0,1.0
62863,Z,0.0,1.0
62864,Zafehouse Diaries,0.0,1.0
62865,Zeno Clash 2,0.0,1.0


In [60]:
r5 = o.recommend(users=['58345543'])
r5

UserID,Game,score,rank
58345543,Combat Arms,0.02556089524392251,1
58345543,Dragon Nest Europe,0.022654426169467043,2
58345543,Global Agenda,0.020009458691508206,3
58345543,Mark of the Ninja,0.017079286031178885,4
58345543,Batman Arkham City GOTY,0.016511627951183834,5
58345543,Batman Arkham Asylum GOTYEdition ...,0.01641233386220159,6
58345543,Frozen Synapse,0.016036841604444716,7
58345543,FTL Faster Than Light,0.015591508364892221,8
58345543,Dear Esther,0.01530916178906644,9
58345543,Dishonored,0.015285215399286768,10


In [61]:
clean_df.loc[clean_df["UserID"] == 58345543,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
115598,Spiral Knights,70.0,10.0
115599,Blacklight Retribution,64.0,10.0
115600,Dungeon Defenders,56.0,8.0
115601,Terraria,54.0,8.0
115602,Borderlands 2,53.0,8.0
...,...,...,...
116260,Wrath of Athena,0.0,1.0
116261,WTFast Gamers Private Network (GPN),0.0,1.0
116262,Xam,0.0,1.0
116263,You Have to Win the Game,0.0,1.0
